# Create and orchestrate NLP workflow using SageMaker Pipelines 

## Contents

## 1. Setup

In [1]:
%%capture

!pip install --upgrade sagemaker

### Imports 

In [2]:
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.steps import TrainingStep
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.huggingface import HuggingFace
from sagemaker.inputs import TrainingInput
from time import gmtime, strftime
from pprint import pprint
import pandas as pd
import sagemaker
import logging
import boto3
import os

In [3]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [4]:
logger.info(f'Using SageMaker: {sagemaker.__version__}')

Using SageMaker: 2.49.0


In [5]:
session = sagemaker.Session()
bucket = session.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

current_timestamp = strftime('%m-%d-%H-%M', gmtime())
pipeline_name = f'nlp-pipeline-{current_timestamp}'

In [6]:
logger.info(f'Bucket name = {bucket}')
logger.info(f'Role = {role}')
logger.info(f'Region = {region}')

Bucket name = sagemaker-us-east-1-892313895307
Role = arn:aws:iam::892313895307:role/service-role/AmazonSageMaker-ExecutionRole-20210714T091788
Region = us-east-1


## Define pipeline parameters 

In [7]:
training_instance_count = ParameterInteger(name='TrainingInstanceCount', default_value=1)
training_instance_type = ParameterString(name='TrainingInstanceType', default_value='ml.p3.2xlarge')
trained_model_s3_uri = ParameterString(name='TrainedModelS3Uri', default_value=f's3://{bucket}/pipeline/model')

## Define training step

In [8]:
hyperparameters={'epochs': 1,
                 'train_batch_size': 16,
                 'model_name':'distilbert-base-uncased',
                 'model_s3': trained_model_s3_uri.default_value,
                 'output_dir':'/opt/ml/checkpoints'}

In [9]:
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./src',
                            instance_type=training_instance_type.default_value,
                            instance_count=training_instance_count.default_value,
                            role=role,
                            transformers_version='4.6',
                            tensorflow_version='2.4',
                            py_version='py37',  
                            disable_profiler=True,
                            debugger_hook_config=False,
                            #model_dir=trained_model_s3_uri.default_value,
                            #output_dir=trained_model_s3_uri.default_value,
                            #output_path=trained_model_s3_uri.default_value,
                            checkpoint_s3_uri=trained_model_s3_uri.default_value,
                            hyperparameters=hyperparameters)

In [10]:
training_step = TrainingStep(
    name='train',
    estimator=huggingface_estimator
)

training_step

TrainingStep(name='train', step_type=<StepTypeEnum.TRAINING: 'Training'>, depends_on=None)

In [11]:
training_step.properties.ModelArtifacts.S3ModelArtifacts.__dict__

{'_path': 'Steps.train.ModelArtifacts.S3ModelArtifacts',
 '_shape_names': ['S3Uri'],
 '__str__': 'S3Uri'}

## Processing Step

In [12]:
"""
endpoint_name = "hf-clf-" + strftime('%d-%H-%M-%S', gmtime())
deploy_model_script_uri = f's3://{default_bucket}/{prefix}/code/deploy_model.py'


s3_client.upload_file(Filename='./src/deploy_model.py', Bucket=default_bucket, Key=f'{prefix}/code/deploy_model.py')

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role=sagemaker_role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name='deploy-model-processing-job',
    sagemaker_session=sagemaker_session)

deploy_step = ProcessingStep(
    name='DeployModel', 
    processor=deploy_model_processor,
    job_arguments=[
        "--model-name", endpoint_name, # just reusing endpoint name here 
        "--region", region,
        "--endpoint-instance-type", deploy_model_instance_type,
        "--model-s3-path", model_s3_path, 
        "--endpoint-name", endpoint_name],
    code=deploy_model_script_uri)
    """

'\nendpoint_name = "hf-clf-" + strftime(\'%d-%H-%M-%S\', gmtime())\ndeploy_model_script_uri = f\'s3://{default_bucket}/{prefix}/code/deploy_model.py\'\n\n\ns3_client.upload_file(Filename=\'./src/deploy_model.py\', Bucket=default_bucket, Key=f\'{prefix}/code/deploy_model.py\')\n\ndeploy_model_processor = SKLearnProcessor(\n    framework_version=\'0.23-1\',\n    role=sagemaker_role,\n    instance_type="ml.t3.medium",\n    instance_count=1,\n    base_job_name=\'deploy-model-processing-job\',\n    sagemaker_session=sagemaker_session)\n\ndeploy_step = ProcessingStep(\n    name=\'DeployModel\', \n    processor=deploy_model_processor,\n    job_arguments=[\n        "--model-name", endpoint_name, # just reusing endpoint name here \n        "--region", region,\n        "--endpoint-instance-type", deploy_model_instance_type,\n        "--model-s3-path", model_s3_path, \n        "--endpoint-name", endpoint_name],\n    code=deploy_model_script_uri)\n    '

## Create Pipeline

In [13]:
pipeline = Pipeline(
    name=pipeline_name,
    steps=[training_step],
    sagemaker_session=session)

In [14]:
pipeline.__dict__

{'name': 'nlp-pipeline-07-18-16-54',
 'parameters': [],
 'pipeline_experiment_config': <sagemaker.workflow.pipeline_experiment_config.PipelineExperimentConfig at 0x7fd257d13710>,
 'steps': [TrainingStep(name='train', step_type=<StepTypeEnum.TRAINING: 'Training'>, depends_on=None)],
 'sagemaker_session': <sagemaker.session.Session at 0x7fd257cc7210>}

In [15]:
response = pipeline.create(role_arn=role)
response

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:892313895307:pipeline/nlp-pipeline-07-18-16-54',
 'ResponseMetadata': {'RequestId': 'f63484d5-b7f5-4d02-9730-c7ce3da5faa5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f63484d5-b7f5-4d02-9730-c7ce3da5faa5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Sun, 18 Jul 2021 16:54:41 GMT'},
  'RetryAttempts': 0}}

In [16]:
execution = pipeline.start()

In [17]:
execution.arn

'arn:aws:sagemaker:us-east-1:892313895307:pipeline/nlp-pipeline-07-18-16-54/execution/atehf72d087i'

In [18]:
status = execution.describe()
pprint(status)

{'CreatedBy': {'DomainId': 'd-dowart1jabkf',
               'UserProfileArn': 'arn:aws:sagemaker:us-east-1:892313895307:user-profile/d-dowart1jabkf/ts-zd-e2e',
               'UserProfileName': 'ts-zd-e2e'},
 'CreationTime': datetime.datetime(2021, 7, 18, 16, 54, 42, 66000, tzinfo=tzlocal()),
 'LastModifiedBy': {'DomainId': 'd-dowart1jabkf',
                    'UserProfileArn': 'arn:aws:sagemaker:us-east-1:892313895307:user-profile/d-dowart1jabkf/ts-zd-e2e',
                    'UserProfileName': 'ts-zd-e2e'},
 'LastModifiedTime': datetime.datetime(2021, 7, 18, 16, 54, 42, 66000, tzinfo=tzlocal()),
 'PipelineArn': 'arn:aws:sagemaker:us-east-1:892313895307:pipeline/nlp-pipeline-07-18-16-54',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:892313895307:pipeline/nlp-pipeline-07-18-16-54/execution/atehf72d087i',
 'PipelineExecutionDisplayName': 'execution-1626627282196',
 'PipelineExecutionStatus': 'Executing',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '723',
         